We will split the notebook into the following five sections:
1. Data preparation
2. Set up MPI processes
3. Applying distributed kernel ridge regression
4. Obtaining predicted median value using kernel function
5. Evaluating model performance
6. Cross Validation / Model Tuning


In [ ]:
# Install dependencies
!pip3 install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip3 install mpi4py

# Import Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from mpi4py import MPI

## **Data Preparation**

In [ ]:
#Import Data
df = pd.read_csv('housing.tsv', sep='\t', header=None)


In [ ]:
#Import

df.columns = ['longitude', 'latitude', 'housingMedianAge', 'totalRooms',
              'totalBedrooms', 'population', 'households', 'medianIncome',
              'oceanProximity', 'medianHouseValue']

              
# Display basic information
print(df.info())
print("\nFirst few rows:")
print(df.head())


In [ ]:
#Normalize Data

features = ['longitude', 'latitude', 'housingMedianAge', 'totalRooms',
              'totalBedrooms', 'population', 'households', 'medianIncome',
              'oceanProximity', 'medianHouseValue']

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

print("\nDataset after preprocessing:")
print(df.head())

In [ ]:
#Split Data

X = df[features]
y = df['medianHouseValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

## **Set up MPI Processes**

In [ ]:
# Set up MPI communicator, rank, etc.

comm = MPI.COMM_WORLD       #Initializes the communicator for all parallel processes in the MPI environment
rank = comm.Get_rank()      #This command gets the rank of the current process
size= comm.Get_size()       #This command gets the total number of processes that are running

In [ ]:
# Data distribution into parallel processes, pt.1 - split the data

if rank == 0:                                               #On the master process, process 0, let's handle the initial data distribution
    X_chunks = np.array_split(X_train, size)                #Split data by number of processes (a.k.a. size)
    y_chunks = np.array_split(y_train, size)
else:
    X_chunks = None                                         #All other processes do not have access to the full dataset at the start.
    y_chunks = None


In [ ]:
print(X_chunks)

In [ ]:
# Data distribution into parallel processes, pt.2 - scatter the data

X_local = comm.scatter(X_chunks, root=0)    #X_local is the portion of X_chunks allocated to the local process
y_local = comm.scatter(y_chunks, root=0)    #Same for y_local

In [ ]:
print(y_local)

## **Applying Kernel Ridge Regression**


In [ ]:
#Define Kernel Computation Function
def compute_gaussian_kernel(X1, X2, sigma):
    X1 = np.array(X1)
    X2 = np.array(X2)
    dists = np.sum(X1**2, axis=1).reshape(-1, 1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T) # |x1-x2|^2 = (x1)^2 + (x2)^2 -2(x1 . x2)
    return np.exp(-dists / (2 * sigma ** 2))                                                    # exp( - dists / 2.sigma )


In [ ]:
#Circular communication for kernel computation. 
'''
Meaning, iteratively, process 0 dot process 0, then process 0 dot process 1, then process 0 dot process 2, then append them together. 
That's process 0's total output row of kernel compute. 
We need to send the data from each process to another process in a circular way, so each process gets a new chunk of unseen data to dot with.
'''

kernel_matrix_local = np.zeros((X_local.shape[0], size)) #Each process will build a row of the full kernel matrix (of size x size)

#Circular Communication for kernel computation

for round in range(size):
    if round == 0:                   #In round 0, each process will compare with itself
        X_other_local = X_local
    else: 
        X_other_local = comm.sendrecv(X_local, dest=(rank-1)%size, source=(rank+1)%size)   
        '''
        Remember that modulo is defined as: a % b = a - (a // b) * b
        #But in python, the floor function for a negative a //b makes it more negative. This elicits a certain circular behaviour:
        #Eg. in round 1, if you have 4 total processes, process 0 must send to process 3. 
        #Destination for process 0: (0-1)%4 = -1%4 = (-1) - (-1)(4) = 3!
        '''

    #Compute the kernel value (dot product of two local data chunks)
    kernel_value = compute_gaussian_kernel(X_local, X_other_local, sigma=1.0)

    #print(kernel_value)

    #Determine where to put this local kernel_value in the final output row for each process
    index = (rank + round) % size
    kernel_matrix_local[:, index] = np.diagonal(kernel_value)


In [ ]:
#Let's do regularization and conjugate gradient computation in parallel. So we will put off mpi.gather till later.

'''

#Gather all the rows back to form the full kernel matrix K at root process 0

if rank == 0:
    full_kernel_matrix = np.zeroes((X_local.shape[0] * size, size))
else:
    full_kernel_matrix = None

#MPI gather 

comm.Gather(kernel_matrix_local, full_kernel_matrix, root = 0)

if rank == 0:
    print(full_kernel_matrix)
'''

In [ ]:
#Obtain distributed matrix A

# kernel_matrix_local is the distributed portion of K for each process
# Add regularization term lambda * I locally

lambda_value = 0.1  #Temporary lambda value

# Assuming each process has a chunk of the matrix (rows)
# global_start_index is the starting row index for each process in the global matrix
global_start_index = rank * kernel_matrix_local.shape[0]  # e.g., rank 0 handles rows 0,1; rank 1 handles rows 2,3, etc.

for i in range(kernel_matrix_local.shape[0]):
    global_index = global_start_index + i  # Compute the global diagonal index
    kernel_matrix_local[i, i] += lambda_value if global_index == global_start_index + i else 0



In [ ]:
#Solve for alpha, using A(alpha) = y, and the Conjugate Gradient Method

# Matrix-vector multiplication of A * p
def mv_mul(A_local, p_global, comm):
    #Each process computes its local contribution to the matrix-vector multiplication
    p_local = p_global[rank * A_local.shape[0] : (rank+1) * A_local.shape[0]]    #P is the direction of movement for our solution
    w_local = np.dot(A_local, p_local)

    #Gather results from all processes to form the full result vector w_global
    w_global = np.zeros_like(p_global)
    comm.Allgather(w_local, w_global)

    return w_global


# Inner product for (r.T * r) or (p.T * w), using MPI_Allreduce for global sum
def inner_product(v_local1, v_local2, comm):
    local_dot_product = np.dot(v_local1, v_local2)
    global_dot_product = comm.allreduce(local_dot_product, op=MPI.SUM)
    return global_dot_product


# Compute conjugate gradient algorithm process, returns alpha_local 
def conjugate_gradient_distributed(K_local, y_local, tol=1e-5, max_iter=1000):

    #Initialize alpha (solution) locally as zero
    alpha_local = np.zeros_like(y_local)

    #Initialize residual, r = y-K @ alpha. Initially alpha is 0, so r = y
    r_local = y_local.copy()

    #Initialize direction p = r
    p_local = r_local.copy()

    #Compute initial squared norm of the residual r
    SE = inner_product(r_local, r_local, comm)

    #Start iterations
    for iteration in range(max_iter):
        if rank == 0:
            print(f"Iteration {iteration}, residual norm: {np.sqrt(SE)}")
        if np.sqrt(SE) < tol:
            break

        #Matrix vector product of w = K @ p
        w_global = mv_mul(K_local, p_local, comm)

        #Compute dot product p.T w
        s = inner_product(p_local, w_local, comm)

        #Update step size: alpha_Step = (r.T r)/(p.T K p)
        alpha_step = SE / s

        #Update alpha: alpha = alpha + alpha_step * p
        alpha_local += alpha_step * p_local

        #Update residual: r = r - alpha_step * w
        r_local -= alpha_step * w_global

        #Compute new squared error/squared norm of residual r
        new_SE = inner_product(r_local, r_local, comm)

        #Compute new beta
        beta = new_SE / SE

        #Update direction p: p = r + beta * p
        p_local = r_local + beta * p_local

        #Update SE variable
        SE = new_SE

    #Gather results from all processes to generate alpha
    #alpha_global = np.hstack(comm.allgather(alpha_local))

    return alpha_local

In [ ]:
alpha_local = conjugate_gradient_distributed(kernel_matrix_local, y_local)

In [ ]:
#Our estimator function f(x) is defined as SUM to N of (alpha_i . k(x,x_i) ). We use that to predict median value!

def f_estimator(X, X_local, alpha_local, comm):

    #Broadcast X to all processes
    X = comm.bcast(X, root = 0)

    #Number of training points N
    N = alpha_global.shape[0]

    #Initialize the local sum
    local_sum = 0.0

    #Compute local sum of alpha_i . K(x, x_i)
    for i in range(X_local.shape[0]):
        
        local_sum += alpha_global[i] * compute_gaussian_kernel(X, X_local[i], sigma=1.0)  #Not sure, feel like this is wrong, we need to find for indiv x value

    #Use MPI_reduce to sum up results in all processes
    global_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

    return global_sum


## **Obtaining predicted median value using kernel function / assess model performance**

In [ ]:
def compute_rmse(f_estimator, y_test, x_test, alpha_global, comm):

    N = len(y_test)
    f_values = np.zeros(N)

    for i in range(N):
        x_i = x_test[i]
        f_values[i] = f_estimator(x_test, x_i, alpha_global, comm)

    rmse = np.sqrt( np.mean(f_values - y_training) **2 )

    return rmse



In [ ]:
if rank == 0:
    rmse = compute_rmse(f_estimator, y_test, x_test, alpha_global, comm)
    print(rmse)
